In [1]:
%pip install snscrape

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import json
import pandas as pd
import subprocess
import snscrape
from datetime import datetime

In [ ]:
import os
import snscrape
import json
import pandas as pd
import subprocess
from datetime import datetime, timedelta
import time
from typing import List

class TelegramParser:
    def __init__(self, output_file: str = "telegram_data.csv"):
        self.output_file = output_file
        self.df = pd.DataFrame(columns=['text', 'title', 'url', 'source', 'date'])
        self._load_existing_data()
        
    def _load_existing_data(self):
        """Загрузка существующих данных из CSV"""
        if os.path.exists(self.output_file):
            self.df = pd.read_csv(self.output_file)
            self.df['date'] = pd.to_datetime(self.df['date'])
            
    def _parse_channel(self, channel: str, start_date: str = None) -> pd.DataFrame:
        """Парсинг одного канала"""
        try:
            # Формируем команду для snscrape
            cmd = ["snscrape", "--jsonl", "telegram-channel", channel]
            
            if start_date:
                cmd.extend(["--since", start_date])
                
            # Запускаем процесс и получаем вывод
            result = subprocess.run(
                cmd,
                check=True,
                capture_output=True,
                text=True,
                timeout=300
            )
            
            # Парсим результат
            posts = []
            for line in result.stdout.split('\n'):
                if not line.strip():
                    continue
                try:
                    data = json.loads(line)
                    post = {
                        'text': data.get('content', ''),
                        'title': f"{channel}_{data.get('id', '')}",
                        'url': data.get('url', ''),
                        'date': data.get('date', ''),
                        'source': f"tg_{channel}"
                    }
                    posts.append(post)
                except json.JSONDecodeError:
                    continue
                    
            return pd.DataFrame(posts)
            
        except (subprocess.CalledProcessError, subprocess.TimeoutExpired) as e:
            print(f"Ошибка парсинга {channel}: {str(e)}")
            return pd.DataFrame()

    def _update_data(self, new_data: pd.DataFrame):
        """Обновление основного DataFrame"""
        if not new_data.empty:
            # Конвертируем даты
            new_data['date'] = pd.to_datetime(new_data['date'])
            
            # Объединяем данные и удаляем дубликаты
            self.df = pd.concat([self.df, new_data], ignore_index=True)
            self.df = self.df.drop_duplicates(subset='url', keep='first')
            
            # Сохраняем в файл
            self.df.to_csv(self.output_file, index=False)
            
    def parse_channels(self, channels: List[str], start_date: str = None):
        """Парсинг списка каналов"""
        for channel in channels:
            print(f"Парсинг канала: {channel}")
            channel_data = self._parse_channel(channel, start_date)
            self._update_data(channel_data)
            time.sleep(1)  # Задержка между запросами
            
    def start_periodic_parsing(self, channels: List[str], interval: int = 600):
        """Запуск периодического парсинга"""
        while True:
            print(f"Начало нового цикла парсинга в {datetime.now()}")
            self.parse_channels(channels)
            print(f"Следующий цикл через {interval//60} минут...")
            time.sleep(interval)

    def get_last_date(self) -> str:
        """Получение последней даты в данных"""
        if not self.df.empty:
            return self.df['date'].max().strftime('%Y-%m-%d')
        return None

# Пример использования
if __name__ == "__main__":
    parser = TelegramParser()
    channels = ["investokrat", 'tb_invest_official', 'alfa_investments', 'Coin_Post']
    
    # Первоначальный парсинг за последние 7 дней
    start_date = (datetime.now() - timedelta(days=1)).strftime("%Y.%m.%d")
    parser.parse_channels(channels)
    
    # Запуск периодического парсинга каждые 10 минут
    # parser.start_periodic_parsing(channels)

Парсинг канала: investokrat
Парсинг канала: tb_invest_official
Парсинг канала: alfa_investments
Парсинг канала: Coin_Post


In [ ]:
data = pd.read_csv('telegram_data.csv')

In [ ]:
data

,text,title,url,source,date
0,NaN,investokrat_,https://t.me/s/investokrat/2634,tg_investokrat,2025-06-08 08:10:23+00:00
1,NaN,investokrat_,https://t.me/s/investokrat/2633,tg_investokrat,2025-06-08 08:08:29+00:00
2,Fusion Factor выкупает АДР NanduQ (QIWI) на М...,investokrat_,https://t.me/s/investokrat/2632,tg_investokrat,2025-06-06 09:01:04+00:00
3,💛 #BTC: Новый фьючерс от Мосбиржи на биткоинС ...,investokrat_,https://t.me/s/investokrat/2631,tg_investokrat,2025-06-06 05:05:58+00:00
4,💼 Программа долгосрочных сбережений - от А до ...,investokrat_,https://t.me/s/investokrat/2629,tg_investokrat,2025-06-05 04:10:04+00:00
...,...,...,...,...,...
15270,Сегодня предлагаем присмотреться к газовому се...,alfa_investments_,https://t.me/s/alfa_investments/14,tg_alfa_investments,2021-06-29 13:17:00+00:00
15271,NaN,alfa_investments_,https://t.me/s/alfa_investments/13,tg_alfa_investments,2021-06-29 07:45:51+00:00
15272,"Как быстро понять, что происходит на рынках? П...",alfa_investments_,https://t.me/s/alfa_investments/12,tg_alfa_investments,2021-06-29 07:45:16+00:00
15273,Давайте знакомиться 👋Привет! Это команда Альфа...,alfa_investments_,https://t.me/s/alfa_investments/9,tg_alfa_investments,2021-06-29 05:37:41+00:00
